# 🔄 GitHub 모델을 활용한 기본 에이전트 워크플로우 (Python)

## 📋 워크플로우 오케스트레이션 튜토리얼

이 노트북은 Microsoft Agent Framework의 강력한 **워크플로우 빌더** 기능을 소개합니다. 복잡한 비즈니스 프로세스를 처리하고 여러 AI 작업을 원활하게 조정할 수 있는 정교한 다단계 에이전트 워크플로우를 만드는 방법을 배워보세요.

## 🎯 학습 목표

### 🏗️ **워크플로우 아키텍처**
- **워크플로우 빌더**: 복잡한 다단계 프로세스를 설계하고 오케스트레이션
- **이벤트 기반 실행**: 워크플로우 이벤트와 상태 전환 처리
- **시각적 워크플로우 디자인**: 워크플로우 구조를 생성하고 시각화
- **GitHub 모델 통합**: 워크플로우 컨텍스트 내에서 AI 모델 활용

### 🔄 **프로세스 오케스트레이션**
- **순차적 작업**: 여러 에이전트 작업을 논리적 순서로 연결
- **조건부 로직**: 의사 결정 지점과 분기 워크플로우 구현
- **오류 처리**: 강력한 오류 복구 및 워크플로우 복원력
- **상태 관리**: 워크플로우 실행 상태 추적 및 관리

### 📊 **엔터프라이즈 워크플로우 패턴**
- **비즈니스 프로세스 자동화**: 복잡한 조직 워크플로우 자동화
- **다중 에이전트 조정**: 여러 전문 에이전트 간 조정
- **확장 가능한 실행**: 엔터프라이즈 규모의 작업을 위한 워크플로우 설계
- **모니터링 및 관찰성**: 워크플로우 성능 및 결과 추적

## ⚙️ 사전 준비 및 설정

### 📦 **필수 종속성**

워크플로우 기능이 포함된 Agent Framework 설치:

```bash
pip install agent-framework-core -U
```

### 🔑 **GitHub 모델 설정**

**환경 설정 (.env 파일):**
```env
GITHUB_TOKEN=your_github_personal_access_token
GITHUB_ENDPOINT=https://models.inference.ai.azure.com
GITHUB_MODEL_ID=gpt-4o-mini
```

### 🏢 **엔터프라이즈 사용 사례**

**비즈니스 프로세스 예시:**
- **고객 온보딩**: 다단계 검증 및 설정 워크플로우
- **콘텐츠 파이프라인**: 자동화된 콘텐츠 생성, 검토 및 게시
- **데이터 처리**: AI 기반 변환을 포함한 ETL 워크플로우
- **품질 보증**: 자동화된 테스트 및 검증 프로세스

**워크플로우의 장점:**
- 🎯 **신뢰성**: 오류 복구가 가능한 결정론적 실행
- 📈 **확장성**: 대량 프로세스 자동화 처리
- 🔍 **관찰성**: 완전한 감사 기록 및 모니터링
- 🔧 **유지보수성**: 시각적 설계 및 모듈형 구성 요소

## 🎨 워크플로우 디자인 패턴

### 기본 워크플로우 구조
```mermaid
graph TD
    A[Start] --> B[Agent Task 1]
    B --> C{Decision Point}
    C -->|Success| D[Agent Task 2]
    C -->|Failure| E[Error Handler]
    D --> F[End]
    E --> F
```

**핵심 구성 요소:**
- **WorkflowBuilder**: 주요 오케스트레이션 엔진
- **WorkflowEvent**: 이벤트 처리 및 통신
- **WorkflowViz**: 시각적 워크플로우 표현 및 디버깅

첫 번째 지능형 워크플로우를 만들어봅시다! 🚀


In [2]:
! pip install agent-framework-core -U


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
# 🔄 Import Workflow and Agent Framework Components
# Core components for building sophisticated agent workflows

from agent_framework.openai import OpenAIChatClient    # 🤖 GitHub Models client integration
from agent_framework import WorkflowBuilder, WorkflowEvent, WorkflowViz  # 🏗️ Workflow orchestration tools

In [4]:
# 📦 Import Environment and System Utilities
# Essential libraries for configuration and environment management

import os                      # 🔧 Environment variable access
from dotenv import load_dotenv # 📁 Secure configuration loading

In [5]:
# 🔧 Initialize Environment Configuration
# Load GitHub Models API credentials from .env file
load_dotenv()

True

In [6]:
# 🔗 Initialize GitHub Models Chat Client for Workflow Operations
# Create the AI client that will power agents within our workflow
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"),    # 🌐 GitHub Models API endpoint
    api_key=os.environ.get("GITHUB_TOKEN"),        # 🔑 Authentication token
    model_id=os.environ.get("GITHUB_MODEL_ID")  # 🎯 Selected AI model
)

ServiceInitializationError: OpenAI model ID is required. Set via 'model_id' parameter or 'OPENAI_CHAT_MODEL_ID' environment variable.

In [ ]:
REVIEWER_NAME = "Concierge"
REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """

In [ ]:
FRONTDESK_NAME = "FrontDesk"
FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """

In [ ]:
reviewer_agent   = chat_client.create_agent(
        instructions=(
           REVIEWER_INSTRUCTIONS
        ),
        name=REVIEWER_NAME,
    )

front_desk_agent = chat_client.create_agent(
        instructions=(
            FRONTDESK_INSTRUCTIONS
        ),
        name=FRONTDESK_NAME,
    )

In [ ]:
workflow = WorkflowBuilder().set_start_executor(front_desk_agent).add_edge(front_desk_agent, reviewer_agent).build()

In [ ]:

print("Generating workflow visualization...")
viz = WorkflowViz(workflow)
# Print out the mermaid string.
print("Mermaid string: \n=======")
print(viz.to_mermaid())
print("=======")
# Print out the DiGraph string.
print("DiGraph string: \n=======")
print(viz.to_digraph())
print("=======")
svg_file = viz.export(format="svg")
print(f"SVG file saved to: {svg_file}")

In [ ]:
class DatabaseEvent(WorkflowEvent): ...

In [ ]:
# Display the exported workflow SVG inline in the notebook

from IPython.display import SVG, display, HTML
import os

print(f"Attempting to display SVG file at: {svg_file}")

if svg_file and os.path.exists(svg_file):
    try:
        # Preferred: direct SVG rendering
        display(SVG(filename=svg_file))
    except Exception as e:
        print(f"⚠️ Direct SVG render failed: {e}. Falling back to raw HTML.")
        try:
            with open(svg_file, "r", encoding="utf-8") as f:
                svg_text = f.read()
            display(HTML(svg_text))
        except Exception as inner:
            print(f"❌ Fallback HTML render also failed: {inner}")
else:
    print("❌ SVG file not found. Ensure viz.export(format='svg') ran successfully.")


In [ ]:
result =''
async for event in workflow.run_stream('I would like to go to Paris.'):
    if isinstance(event, DatabaseEvent):
        print(f"{event}")
    if isinstance(event, WorkflowEvent):
        result += str(event.data)
        # print(f"Workflow output: {event.data}")

In [ ]:
result.replace("None", "")


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서(원어로 작성된 문서)를 신뢰할 수 있는 권위 있는 자료로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
